<a href="https://colab.research.google.com/github/brianpenrod/project_alpha_venezuela_risk/blob/main/PROJECT_ALPHA_FIGURE_KIT_(AUDIT_GRADE%2C_REPRODUCIBLE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# PROJECT ALPHA - ONE-CELL RUNNABLE FIGURE KIT (CLEAN + AUDITABLE)
# 1) Waterfall: Debt → Control Value (explicit control premium bar)
# 2) Sensitivity Table: recovery 10–40%
# 3) Kinetic: Reaction Windows (Mach 5–10 band; point at Mach 7.5)
# ============================================================

import pandas as pd
import plotly.graph_objects as go

# --------------------------
# A) GLOBAL STYLE
# --------------------------
tier1_template = "plotly_dark"
font_family = "Arial"

# --------------------------
# B) ASSUMPTIONS (EDIT HERE)
# --------------------------
# Forensics (Billions USD)
total_credit_b = 65.0
repaid_b = 51.0
base_rr = 20  # headline recovery-rate scenario (%) shown in the waterfall

recovery_rate_scenarios = [10, 20, 30, 40]

# Kinetic (distances in km)
targets = [
    "Puerto Rico (Air Base)",
    "Panama Canal (Logistics)",
    "Miami (Command Node)",
    "Houston (Energy Node)"
]
dist_km = [900, 1100, 2200, 3300]

# Speed of sound proxy (km/h) — varies with altitude/temperature; treat as scenario input
speed_of_sound_kmh = 1235

mach_min, mach_max = 5, 10
mach_point = 7.5  # plotted dot for readability; band spans Mach 5–10

# --------------------------
# C) HELPERS
# --------------------------
def control_multiplier_from_recovery(rr_pct: float) -> float:
    """
    Scenario heuristic: Multiplier = 1 + (1 - rr)
    rr=20% -> 1.8x
    """
    rr = rr_pct / 100.0
    return 1.0 + (1.0 - rr)

def minutes(distance_km: float, speed_kmh: float) -> float:
    return (distance_km / speed_kmh) * 60.0

# ============================================================
# 1) FORENSICS — WATERFALL (NO marker=; use totals/increasing/decreasing)
# ============================================================
outstanding_b = total_credit_b - repaid_b
mult = control_multiplier_from_recovery(base_rr)

implied_strategic_value_b = outstanding_b * mult
control_premium_b = implied_strategic_value_b - outstanding_b
expected_cash_recovery_b = outstanding_b * (base_rr / 100.0)

fig_waterfall = go.Figure(go.Waterfall(
    orientation="v",

    # Key trick:
    # - "absolute" for Total Credit so it uses totals color
    # - "total" for Outstanding and Final so they use totals color
    # - "relative" for negative repaid (decreasing color)
    # - "relative" for positive premium (increasing color)
    measure=["absolute", "relative", "total", "relative", "total"],

    x=[
        "Total Credit",
        "Principal Repaid",
        "Outstanding Debt",
        "Control Premium (heuristic)",
        "Implied Strategic Value"
    ],

    y=[
        total_credit_b,
        -repaid_b,
        outstanding_b,
        control_premium_b,
        implied_strategic_value_b
    ],

    textposition="outside",
    text=[
        f"${total_credit_b:.1f}B",
        f"-${repaid_b:.1f}B",
        f"${outstanding_b:.1f}B",
        f"+${control_premium_b:.1f}B",
        f"${implied_strategic_value_b:.1f}B (mult={mult:.2f}x)"
    ],

    connector={"line": {"color": "#7f8c8d"}},

    # Color semantics:
    totals={"marker": {"color": "#ffa15a"}},      # neutral/orange for totals & absolute bars
    decreasing={"marker": {"color": "#00cc96"}},  # green for repaid (money in)
    increasing={"marker": {"color": "#636efa"}},  # purple for control premium (model construct)
))

fig_waterfall.update_layout(
    title=dict(
        text=(
            f"<b>FORENSIC SCENARIO: Debt → Control Value (Recovery={base_rr}%)</b>"
            f"<br><span style='font-size:12px;'>"
            f"Strategic value is a scenario heuristic (control premium), not cash recovery. "
            f"Cash recovery @ {base_rr}% = ${expected_cash_recovery_b:.1f}B."
            f"</span>"
        ),
        font=dict(size=20)
    ),
    template=tier1_template,
    showlegend=False,
    height=550,
    font=dict(family=font_family),
    margin=dict(l=60, r=40, t=90, b=60),
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)"
)

fig_waterfall.show()

# ============================================================
# 2) FORENSICS — SENSITIVITY TABLE (PRINTS UNDER FIGURE)
# ============================================================
rows = []
for rr in recovery_rate_scenarios:
    m = control_multiplier_from_recovery(rr)
    implied = outstanding_b * m
    rows.append({
        "recovery_rate_pct": rr,
        "control_multiplier_x": round(m, 2),
        "outstanding_b": round(outstanding_b, 2),
        "implied_strategic_value_b": round(implied, 2),
        "control_premium_b": round(implied - outstanding_b, 2),
        "expected_cash_recovery_b": round(outstanding_b * (rr / 100.0), 2)
    })

sens_df = pd.DataFrame(rows).sort_values("recovery_rate_pct")
print("\nSENSITIVITY (Forensics):")
print(sens_df.to_string(index=False))

# ============================================================
# 3) KINETIC — VERSION B2 (POINT @ Mach 7.5; BAND Mach 5–10)
# ============================================================
df = pd.DataFrame({"Target": targets, "Distance_km": dist_km})

v_min = mach_min * speed_of_sound_kmh
v_max = mach_max * speed_of_sound_kmh
v_pt  = mach_point * speed_of_sound_kmh

df["t_point"] = df["Distance_km"].apply(lambda d: minutes(d, v_pt))
df["t_fast"]  = df["Distance_km"].apply(lambda d: minutes(d, v_max))  # Mach 10
df["t_slow"]  = df["Distance_km"].apply(lambda d: minutes(d, v_min))  # Mach 5

# Gate definition: exposed if fastest-case (Mach 10) is within 15 minutes
df["decision_flag"] = df["t_fast"] <= 15

df_exposed = df[df["decision_flag"]].copy()
df_ok      = df[~df["decision_flag"]].copy()

fig_kinetic = go.Figure()

def add_trace(subdf, name):
    fig_kinetic.add_trace(go.Scatter(
        x=subdf["Distance_km"],
        y=subdf["t_point"],
        mode="markers",
        marker=dict(size=14, line=dict(width=2, color="white")),
        error_y=dict(
            type="data",
            symmetric=False,
            array=(subdf["t_slow"] - subdf["t_point"]),
            arrayminus=(subdf["t_point"] - subdf["t_fast"])
        ),
        name=name,
        hovertemplate=(
            "<b>%{customdata[0]}</b><br>"
            "Distance: %{x:.0f} km<br>"
            f"Time (Mach {mach_point}): " + "%{y:.1f} min<br>"
            f"Fast (Mach {mach_max}): " + "%{customdata[1]:.1f} min<br>"
            f"Slow (Mach {mach_min}): " + "%{customdata[2]:.1f} min<br>"
            "<extra></extra>"
        ),
        customdata=list(zip(subdf["Target"], subdf["t_fast"], subdf["t_slow"]))
    ))

add_trace(df_exposed, f"Decision-compression exposed (Mach {mach_max} ≤ 15m)")
add_trace(df_ok,      f"Not exposed at Mach {mach_max}")

# Thresholds
fig_kinetic.add_hline(
    y=15, line_dash="dot",
    annotation_text="<b>DECISION THRESHOLD (15m)</b>",
    annotation_position="bottom right",
    annotation_font=dict(color="#ef553b", size=12),
    line_color="#ef553b", line_width=2
)
fig_kinetic.add_hline(
    y=30, line_dash="dot",
    annotation_text="<b>PLANNING THRESHOLD (30m)</b>",
    annotation_position="top right",
    annotation_font=dict(size=12),
    line_width=2
)

# Note
fig_kinetic.add_annotation(
    xref="paper", yref="paper", x=0.01, y=0.99,
    text=(
        f"<b>Note:</b> Point = Mach {mach_point}; error bar spans Mach {mach_min}–{mach_max}.<br>"
        "Scenario model (avg velocity + great-circle distance proxy)."
    ),
    showarrow=False, align="left",
    font=dict(size=12)
)

fig_kinetic.update_layout(
    title=f"<b>KINETIC LATENCY: Reaction Windows (Mach {mach_min}–{mach_max})</b>",
    xaxis_title="Distance (km)",
    yaxis_title="Flight Time (Minutes)",
    template=tier1_template,
    font=dict(family=font_family),
    height=550,
    margin=dict(l=80, r=280, t=80, b=70),
    legend=dict(yanchor="top", y=0.98, xanchor="left", x=1.02),
    paper_bgcolor="rgba(0,0,0,0)",
    plot_bgcolor="rgba(0,0,0,0)"
)

fig_kinetic.show()

print("\nKinetic Table (inputs + outputs):")
print(
    df[["Target", "Distance_km", "t_fast", "t_point", "t_slow", "decision_flag"]]
    .rename(columns={
        "t_fast":  f"flight_min_mach{mach_max}",
        "t_point": f"flight_min_mach{mach_point}",
        "t_slow":  f"flight_min_mach{mach_min}"
    })
    .round(2)
    .to_string(index=False)
)


SENSITIVITY (Forensics):
 recovery_rate_pct  control_multiplier_x  outstanding_b  implied_strategic_value_b  control_premium_b  expected_cash_recovery_b
                10                   1.9           14.0                       26.6               12.6                       1.4
                20                   1.8           14.0                       25.2               11.2                       2.8
                30                   1.7           14.0                       23.8                9.8                       4.2
                40                   1.6           14.0                       22.4                8.4                       5.6



Kinetic Table (inputs + outputs):
                  Target  Distance_km  flight_min_mach10  flight_min_mach7.5  flight_min_mach5  decision_flag
  Puerto Rico (Air Base)          900               4.37                5.83              8.74           True
Panama Canal (Logistics)         1100               5.34                7.13             10.69           True
    Miami (Command Node)         2200              10.69               14.25             21.38           True
   Houston (Energy Node)         3300              16.03               21.38             32.06          False
